In [1]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display


# =========================
# Colored Output
# =========================

def print_blue(msg):
    print("\033[94m" + str(msg) + "\033[0m")

def print_red(msg):
    print("\033[91m" + str(msg) + "\033[0m")


# =========================
# Parsing + Input Validation
# =========================

def parse_time_hhmm(value):
    try:
        return datetime.strptime(value.strip(), "%H:%M").time()
    except:
        raise ValueError("Invalid time. Use HH:MM (e.g., 08:30).")

def parse_date_flexible(value):
    patterns = ["%Y-%m-%d", "%Y/%m/%d", "%d/%m/%Y", "%d-%m-%Y"]
    for p in patterns:
        try:
            return datetime.strptime(value.strip(), p).date()
        except:
            pass
    raise ValueError("Invalid date format.")

def ask_nonempty(prompt):
    value = input(prompt).strip()
    if value.lower() == "q":
        raise KeyboardInterrupt
    if not value:
        raise ValueError("This field cannot be empty.")
    return value

def ask_float_positive(prompt):
    value = input(prompt).strip()
    if value.lower() == "q":
        raise KeyboardInterrupt
    try:
        v = float(value)
        if v <= 0:
            raise ValueError
        return v
    except:
        raise ValueError("Enter a valid positive number.")

def ask_int_min(prompt, min_value=1):
    value = input(prompt).strip()
    if value.lower() == "q":
        raise KeyboardInterrupt
    try:
        v = int(value)
        if v < min_value:
            raise ValueError
        return v
    except:
        raise ValueError("Enter a valid integer >= " + str(min_value))


# =========================
# OOP Classes
# =========================

class Medication:
    def __init__(self, name, dosage_mg, times_per_day, first_time, duration_days, start_date, taken_count=0):
        self.name = name
        self.dosage_mg = dosage_mg
        self.times_per_day = times_per_day
        self.first_time = first_time
        self.duration_days = duration_days
        self.start_date = start_date
        self.taken_count = taken_count
        self.validate()

    def validate(self):
        if not self.name:
            raise ValueError("Medication name required.")
        if self.dosage_mg <= 0:
            raise ValueError("Dosage must be positive.")
        if self.times_per_day < 1:
            raise ValueError("Times per day must be >= 1.")
        if self.duration_days < 1:
            raise ValueError("Duration must be >= 1.")

    def total_doses(self):
        return self.times_per_day * self.duration_days

    def remaining_doses(self):
        return max(0, self.total_doses() - self.taken_count)

    def dose_times_for_day(self, day):
        end = self.start_date + timedelta(days=self.duration_days - 1)
        if day < self.start_date or day > end:
            return []

        interval = 24 / self.times_per_day
        base = datetime.combine(day, self.first_time)
        return [base + timedelta(hours=i * interval) for i in range(self.times_per_day)]

    def next_dose_datetime(self, now):
        for i in range(self.duration_days + 1):
            d = now.date() + timedelta(days=i)
            for t in self.dose_times_for_day(d):
                if t >= now:
                    return t
        return None


class MedicationSchedule:
    def __init__(self, patient_name):
        self.patient_name = patient_name
        self.items = []

    def add(self, med):
        if any(m.name.lower() == med.name.lower() for m in self.items):
            raise ValueError("Medication already exists.")
        self.items.append(med)

    def find(self, name):
        for m in self.items:
            if m.name.lower() == name.lower():
                return m
        return None

    def remove(self, name):
        before = len(self.items)
        self.items = [m for m in self.items if m.name.lower() != name.lower()]
        return len(self.items) != before


class Reminder:
    def __init__(self, schedule):
        self.schedule = schedule

    def check_due(self, now, window_minutes=15):
        rows = []
        for m in self.schedule.items:
            nxt = m.next_dose_datetime(now)
            if nxt:
                delta = (np.datetime64(nxt) - np.datetime64(now)) / np.timedelta64(1, "m")
                if 0 <= delta <= window_minutes:
                    rows.append({
                        "Medication": m.name,
                        "Next Dose": nxt.strftime("%Y-%m-%d %H:%M"),
                        "Minutes Left": int(delta)
                    })
        return pd.DataFrame(rows)

    def daily_report(self, day):
        rows = []
        for m in self.schedule.items:
            for t in m.dose_times_for_day(day):
                rows.append({
                    "Medication": m.name,
                    "Dosage (mg)": m.dosage_mg,
                    "Dose Time": t.strftime("%H:%M"),
                    "Remaining Doses": m.remaining_doses()
                })
        df = pd.DataFrame(rows)
        if not df.empty:
            df = df.sort_values(by="Dose Time")
        return df


# =========================
# DataFrame Output
# =========================

def build_patient_dataframe(schedule):
    rows = []
    for m in schedule.items:
        rows.append({
            "Patient": schedule.patient_name,
            "Medication": m.name,
            "Dosage (mg)": m.dosage_mg,
            "Times/Day": m.times_per_day,
            "First Time": m.first_time.strftime("%H:%M"),
            "Start Date": m.start_date.strftime("%Y-%m-%d"),
            "Duration (days)": m.duration_days,
            "Taken": m.taken_count,
            "Remaining": m.remaining_doses()
        })
    return pd.DataFrame(rows)

def build_all_patients_dataframe(patients_dict):
    frames = []
    for _, sch in patients_dict.items():
        frames.append(build_patient_dataframe(sch))
    if not frames:
        return pd.DataFrame()
    return pd.concat(frames, ignore_index=True)


# =========================
# Visualization (Matplotlib)
# =========================

def plot_remaining_doses(schedule):
    if not schedule.items:
        print_red("No medications to visualize.")
        return

    names = [m.name for m in schedule.items]
    remaining = [m.remaining_doses() for m in schedule.items]

    plt.figure(figsize=(8, 4))
    plt.bar(names, remaining)
    plt.title("Remaining Doses - " + schedule.patient_name)
    plt.xlabel("Medication")
    plt.ylabel("Remaining Doses")
    plt.tight_layout()
    plt.show()


# =========================
# CSV Save/Load
# =========================

CSV_FILE = "medications.csv"

def save_to_csv():
    df = build_all_patients_dataframe(patients)
    if df.empty:
        print_red("Nothing to save.")
        return
    df.to_csv(CSV_FILE, index=False)
    print_blue("Saved to " + CSV_FILE)

def load_from_csv():
    global patients, current_patient

    try:
        df = pd.read_csv(CSV_FILE)
    except FileNotFoundError:
        print_red("CSV file not found: " + CSV_FILE)
        return
    except Exception as e:
        print_red(e)
        return

    if df.empty:
        print_red("CSV is empty.")
        return

    required_cols = [
        "Patient", "Medication", "Dosage (mg)", "Times/Day",
        "First Time", "Start Date", "Duration (days)", "Taken"
    ]
    for col in required_cols:
        if col not in df.columns:
            print_red("CSV missing column: " + col)
            return

    patients.clear()

    for pname in df["Patient"].unique():
        sch = MedicationSchedule(pname)
        sub = df[df["Patient"] == pname]

        for _, row in sub.iterrows():
            try:
                med = Medication(
                    str(row["Medication"]),
                    float(row["Dosage (mg)"]),
                    int(row["Times/Day"]),
                    parse_time_hhmm(str(row["First Time"])),
                    int(row["Duration (days)"]),
                    parse_date_flexible(str(row["Start Date"])),
                    int(row["Taken"])
                )
                sch.items.append(med)
            except Exception:
                # skip bad rows but continue loading
                pass

        patients[pname] = sch

    current_patient = list(patients.keys())[0]
    print_blue("Loaded from " + CSV_FILE)
    print_blue("Current patient: " + current_patient)


# =========================
# Menu System
# =========================

patients = {}
current_patient = None


def setup_patient():
    global current_patient
    try:
        name = ask_nonempty("Enter patient name (q to cancel): ")
        if name not in patients:
            patients[name] = MedicationSchedule(name)
            print_blue("New patient created.")
        else:
            print_blue("Switched to existing patient.")
        current_patient = name
    except KeyboardInterrupt:
        print_blue("Cancelled.")

def get_schedule():
    if current_patient is None:
        setup_patient()
    return patients.get(current_patient)

def add_medication():
    sch = get_schedule()
    if not sch:
        return
    try:
        print_blue("Type q to cancel.")
        med = Medication(
            ask_nonempty("Medication name: "),
            ask_float_positive("Dosage (mg): "),
            ask_int_min("Times per day: "),
            parse_time_hhmm(ask_nonempty("First time HH:MM: ")),
            ask_int_min("Duration days: "),
            parse_date_flexible(ask_nonempty("Start date: "))
        )
        sch.add(med)
        print_blue("Medication added successfully.")
    except KeyboardInterrupt:
        print_blue("Cancelled.")
    except Exception as e:
        print_red(e)

def delete_medication():
    sch = get_schedule()
    if not sch:
        return
    try:
        name = ask_nonempty("Medication name to delete (q to cancel): ")
        ok = sch.remove(name)
        if ok:
            print_blue("Medication deleted successfully.")
        else:
            print_red("Medication not found.")
    except KeyboardInterrupt:
        print_blue("Cancelled.")
    except Exception as e:
        print_red(e)


def run_menu():
    setup_patient()
    while True:
        sch = get_schedule()
        if sch is None:
            setup_patient()
            continue

        reminder = Reminder(sch)

        print("\n1 Add medication")
        print("2 Delete medication")
        print("3 Daily report")
        print("4 Due soon")
        print("5 DataFrame (current patient)")
        print("6 DataFrame (all patients)")
        print("7 Switch patient")
        print("8 Save to CSV")
        print("9 Load from CSV")
        print("10 Visualization (Remaining doses)")
        print("0 Exit")

        c = input("Choose: ").strip()
        try:
            if c == "1":
                add_medication()
            elif c == "2":
                delete_medication()
            elif c == "3":
                df = reminder.daily_report(datetime.now().date())
                display(df if not df.empty else "No data")
            elif c == "4":
                df = reminder.check_due(datetime.now())
                display(df if not df.empty else "No due medications")
            elif c == "5":
                dfp = build_patient_dataframe(sch)
                display(dfp if not dfp.empty else "No medications for this patient")
            elif c == "6":
                dfa = build_all_patients_dataframe(patients)
                display(dfa if not dfa.empty else "No patients / medications yet")
            elif c == "7":
                setup_patient()
            elif c == "8":
                save_to_csv()
            elif c == "9":
                load_from_csv()
            elif c == "10":
                plot_remaining_doses(sch)
            elif c == "0":
                print_blue("Bye 👋")
                break
            else:
                print_red("Invalid choice.")
        except Exception as e:
            print_red(e)

run_menu()


Enter patient name (q to cancel):  jana


New patient created.

1 Add medication
2 Delete medication
3 Daily report
4 Due soon
5 DataFrame (current patient)
6 DataFrame (all patients)
7 Switch patient
8 Save to CSV
9 Load from CSV
10 Visualization (Remaining doses)
0 Exit


Choose:  0


Bye 👋
